In [3]:
import pandas as pd

df = pd.read_csv("D:/DATN/US - Accidents/data/raw/US_Accidents_March23.csv")
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 46 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   ID                     object 
 1   Source                 object 
 2   Severity               int64  
 3   Start_Time             object 
 4   End_Time               object 
 5   Start_Lat              float64
 6   Start_Lng              float64
 7   End_Lat                float64
 8   End_Lng                float64
 9   Distance(mi)           float64
 10  Description            object 
 11  Street                 object 
 12  City                   object 
 13  County                 object 
 14  State                  object 
 15  Zipcode                object 
 16  Country                object 
 17  Timezone               object 
 18  Airport_Code           object 
 19  Weather_Timestamp      object 
 20  Temperature(F)         float64
 21  Wind_Chill(F)          float64
 22  Humidity(%)       

In [5]:
print(df.isnull().sum())

ID                             0
Source                         0
Severity                       0
Start_Time                     0
End_Time                       0
Start_Lat                      0
Start_Lng                      0
End_Lat                  3402762
End_Lng                  3402762
Distance(mi)                   0
Description                    5
Street                     10869
City                         253
County                         0
State                          0
Zipcode                     1915
Country                        0
Timezone                    7808
Airport_Code               22635
Weather_Timestamp         120228
Temperature(F)            163853
Wind_Chill(F)            1999019
Humidity(%)               174144
Pressure(in)              140679
Visibility(mi)            177098
Wind_Direction            175206
Wind_Speed(mph)           571233
Precipitation(in)        2203586
Weather_Condition         173459
Amenity                        0
Bump      

In [6]:
print(df['Start_Time'].dtype)


object


In [7]:
print(df['Start_Time'].apply(type).value_counts())


Start_Time
<class 'str'>    7728394
Name: count, dtype: int64


In [4]:
pattern = r"^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$"

invalid_format = ~df["Start_Time"].astype(str).str.match(pattern, na=False)
print(f"Số dòng không đúng định dạng chuẩn: {invalid_format.sum()}")

# In ra một vài dòng lỗi
print(df.loc[invalid_format, "Start_Time"].drop_duplicates().head())


Số dòng không đúng định dạng chuẩn: 743166
3639775    2017-07-23 04:21:01.000000000
3639776    2017-07-23 05:18:46.000000000
3639777    2017-07-23 05:36:01.000000000
3639778    2017-07-23 08:36:01.000000000
3639779    2017-07-23 11:02:16.000000000
Name: Start_Time, dtype: object


In [8]:
import pandas as pd

# Giả sử df là DataFrame ban đầu chưa ép kiểu
invalid_times = df.loc[pd.to_datetime(df["Start_Time"], errors='coerce').isna(), "Start_Time"]

print(f"Số dòng lỗi: {len(invalid_times)}")
print("Ví dụ vài giá trị lỗi:")
print(invalid_times.head(20).tolist())


Số dòng lỗi: 743166
Ví dụ vài giá trị lỗi:
['2017-07-23 04:21:01.000000000', '2017-07-23 05:18:46.000000000', '2017-07-23 05:36:01.000000000', '2017-07-23 08:36:01.000000000', '2017-07-23 11:02:16.000000000', '2017-07-23 13:26:16.000000000', '2017-07-23 13:37:31.000000000', '2017-07-23 13:37:31.000000000', '2017-07-23 13:46:31.000000000', '2017-07-23 14:19:32.000000000', '2017-07-23 14:06:01.000000000', '2017-07-23 14:41:16.000000000', '2017-07-23 14:50:17.000000000', '2017-07-23 14:50:17.000000000', '2017-07-23 15:15:46.000000000', '2017-07-23 15:23:16.000000000', '2017-07-23 17:50:17.000000000', '2017-07-23 17:50:17.000000000', '2017-07-23 18:15:46.000000000', '2017-07-23 18:43:31.000000000']


In [9]:
# Kiểm tra kiểu dữ liệu từng phần tử trong Start_Time
types = df["Start_Time"].apply(type).value_counts()
print(types)


Start_Time
<class 'str'>    7728394
Name: count, dtype: int64


In [10]:
# Chuyển sang string, loại bỏ khoảng trắng đầu cuối
df["Start_Time"] = df["Start_Time"].str.strip()

# Loại bỏ các ký tự không phải số, dấu -, :, space, dấu chấm (dấu phân cách thời gian)
import re

def clean_datetime(s):
    return re.sub(r"[^0-9\-: .]", "", s)

df["Start_Time"] = df["Start_Time"].apply(clean_datetime)

# Thử ép kiểu lại
df["Start_Time_dt"] = pd.to_datetime(df["Start_Time"], errors='coerce')

print("Số dòng lỗi sau khi làm sạch:", df["Start_Time_dt"].isna().sum())


Số dòng lỗi sau khi làm sạch: 743166


In [11]:
def remove_nano_seconds(s):
    # Xoá phần .000000000 cuối chuỗi datetime nếu có
    if isinstance(s, str) and s.endswith(".000000000"):
        return s[:-10]  # bỏ 10 ký tự cuối
    return s

df["Start_Time_clean"] = df["Start_Time"].apply(remove_nano_seconds)

df["Start_Time_dt"] = pd.to_datetime(df["Start_Time_clean"], errors='coerce')

print("Số lỗi sau khi loại bỏ nano giây:", df["Start_Time_dt"].isna().sum())


Số lỗi sau khi loại bỏ nano giây: 60844


In [12]:
# Lấy các dòng không parse được (giá trị NaT)
invalid_dates = df[df["Start_Time_dt"].isna()]["Start_Time_clean"].unique()

print(f"Số dòng lỗi còn lại: {len(invalid_dates)}")
print("Ví dụ vài giá trị lỗi:", invalid_dates[:10])


Số dòng lỗi còn lại: 59202
Ví dụ vài giá trị lỗi: ['2022-04-29 08:25:22.000000' '2022-12-14 06:40:00.000000'
 '2023-01-06 14:01:00.000000' '2022-01-11 14:44:00.000000'
 '2023-01-17 08:39:00.000000' '2022-03-07 10:54:07.000000'
 '2022-07-14 16:55:00.000000' '2022-05-11 15:44:53.000000'
 '2022-12-20 18:30:32.000000' '2022-06-30 16:32:42.000000']


In [13]:
def remove_microseconds(s):
    if isinstance(s, str) and s.endswith(".000000"):
        return s[:-7]  # bỏ 7 ký tự cuối (dấu chấm + 6 số 0)
    return s

df["Start_Time_clean2"] = df["Start_Time_clean"].apply(remove_microseconds)

df["Start_Time_dt2"] = pd.to_datetime(df["Start_Time_clean2"], errors='coerce')

print("Số lỗi sau khi loại bỏ micro giây:", df["Start_Time_dt2"].isna().sum())


Số lỗi sau khi loại bỏ micro giây: 0


In [14]:
import pandas as pd
import re

# Giả sử df là dataframe ban đầu của bạn

def clean_datetime_string(s):
    if isinstance(s, str):
        s = s.strip()
        if s.endswith(".000000000"):
            s = s[:-10]
        elif s.endswith(".000000"):
            s = s[:-7]
        s = re.sub(r"[^0-9\-: .]", "", s)
        s = s.strip()
        return s
    return s

# Áp dụng làm sạch cho End_Time
df["End_Time_clean"] = df["End_Time"].apply(clean_datetime_string)

# Ép kiểu datetime, với errors='coerce' để lỗi thành NaT
df["End_Time_dt"] = pd.to_datetime(df["End_Time_clean"], errors="coerce")

# Tìm các dòng lỗi (NaT)
errors = df[df["End_Time_dt"].isna()]["End_Time"]

print(f"Số dòng lỗi End_Time: {len(errors)}")
print("Ví dụ các giá trị lỗi:")
print(errors.head(20))


Số dòng lỗi End_Time: 439
Ví dụ các giá trị lỗi:
3641213    2023-03-28 15:10:24.045736300
3642911    2023-03-29 07:00:59.599925600
3649487    2023-03-31 23:50:02.638404100
3651430    2023-03-27 14:59:47.318225400
3653741    2023-03-20 15:38:43.238974800
3657432    2023-03-19 07:01:09.763516900
3659099    2023-03-24 00:04:45.328674900
3684052    2022-08-29 21:07:42.897633300
3696989    2022-07-05 21:36:07.437704200
3700898       2022-03-15 22:33:15.766504
3707480    2022-09-19 23:20:12.963182800
3708447    2022-04-25 13:32:18.879480400
3721683    2022-12-22 23:44:00.841032100
3726520    2023-02-15 08:25:07.597343400
3761555    2022-11-03 00:43:29.789616700
3761557    2022-07-28 21:56:32.604635400
3764252    2022-07-02 06:03:28.212801700
3770696    2022-04-07 00:04:16.206993600
3787451    2022-04-22 18:02:15.264367200
3792431    2022-01-25 00:16:46.073114700
Name: End_Time, dtype: object
